## Building an Agent in LangChain
Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

**Building an Agent that can interact with a search engine. We will be able to ask this agent questions and watch it call the search tool and have conversations with it**

In [1]:
# !pip install -U langchain-community langgraph langchain-anthropic tavily-python

In [2]:
# Setting up LangSmith

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [3]:
# Travily for search engine tool.

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [4]:
# Defining tools
# our tool: Tavily search engine

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)
search_results = search.invoke("What is the weather in Pokhara?")
print(search_results)

[{'url': 'https://www.wunderground.com/weather/np/pokhara', 'content': 'Pokhara Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Pokhara area. ... 25 AM GMT+05:45 on June ...'}, {'url': 'https://www.whereandwhen.net/when/central-and-south-asia/nepal/pokhara/june/', 'content': 'Weather in Pokhara in june 2024. The weather in Pokhara in the month of june comes from statistical datas on the last years. You can view the weather statistics for the whole month, but also by using the tabs for the beginning, the middle and the end of the month. ... 25-06-2023 72°F to 83°F. 26-06-2023 72°F to 86°F. 27-06-2023 74°F to 86 ...'}]


In [5]:
# Add more tools if we want to use 

tools = [search]

In [7]:
# Now learning how to use a language model by to call tools.

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [8]:
# Now we can call the language model by passing in a list of messages. By default the resonse is a content string.

from langchain_core.messages import HumanMessage

response = model.invoke(
    [
        HumanMessage(content="Hello, how are you?")
    ]
)

response.content

"I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have! It's great to chat with you. How can I help you today?"

In [9]:
# Binding these language models with our tools

model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke(
    [
        HumanMessage(content="What is the weather in Pokhara Nepal?")
    ]
)

print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_3z3c', 'function': {'arguments': '{"query":"weather in Pokhara Nepal"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 983, 'total_tokens': 1058, 'completion_time': 0.059026771, 'prompt_time': 0.147786325, 'queue_time': None, 'total_time': 0.206813096}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-058db3a2-eb44-4bf1-b2a6-3f43b28ddfec-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Pokhara Nepal'}, 'id': 'call_3z3c'}]


In [13]:
print(f"Content String: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Content String: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Pokhara Nepal'}, 'id': 'call_3z3c'}]


There is not text content but there is a tool call.

This is not calling that tool yer it is just telling us to. 

In order to actually call we need to create agent.

## Creating the agent
now we have our tools and LLM we can create the agent using **LangGraph**.

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# we are passing the model and not the model with tools because create_react_agent will bind the tools to the model for us.

In [18]:
response = agent_executor.invoke(
    {
        "messages": [
            HumanMessage(content="Hi how are you?")
        ]
    }
)

response["messages"]

[HumanMessage(content='Hi how are you?', id='1291d3fd-87e4-4b27-a80b-db911c39534f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6h2d', 'function': {'arguments': '{"query":"Hi how are you?"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 944, 'total_tokens': 1024, 'completion_time': 0.063244924, 'prompt_time': 0.166151737, 'queue_time': None, 'total_time': 0.229396661}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3e8deb7a-3bd7-4017-9ff6-887eb151b3c4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Hi how are you?'}, 'id': 'call_6h2d'}]),
 ToolMessage(content='[{"url": "https://www.fluentu.com/blog/english/how-are-you-esl/", "content": "Say \\u201chello\\u201d to as many people as you can and see how they answer your \\u201cHow are you?\\u201d\\nOnce you\\u2019

watch what is happening in langsmith

https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r

In [19]:
# Trying out an example where it should be invoking the tool

response = agent_executor.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the weather in Berlin?"
            )
        ]
    }
)

response["messages"]

[HumanMessage(content='What is the weather in Berlin?', id='eff807db-658e-49c0-b2f8-408852a5a766'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_regf', 'function': {'arguments': '{"query":"current weather in Berlin"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 980, 'total_tokens': 1053, 'completion_time': 0.05772761, 'prompt_time': 0.148878676, 'queue_time': None, 'total_time': 0.20660628599999997}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a50b2be6-42d2-47e8-859e-551c68cab22c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Berlin'}, 'id': 'call_regf'}]),
 ToolMessage(content='[{"url": "https://world-weather.info/forecast/germany/berlin/june-2024/", "content": "Extended weather forecast in Berlin. Hourly Week 10 days 14 days 30 days Yea

In [20]:
# Streaming messages

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8sb6', 'function': {'arguments': '{"query":"current weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 967, 'total_tokens': 1047, 'completion_time': 0.063344632, 'prompt_time': 0.694802317, 'queue_time': None, 'total_time': 0.7581469489999999}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_c4a72fb330', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f6bed70-bd3b-45a0-a4d4-71b9ccec6f0b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in sf'}, 'id': 'call_8sb6'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://world-weather.info/forecast/usa/san_francisco/june-2024/", "content": "Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed \\u26a1 San Francisco Weather For

In [21]:
# Streamin tokens

async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'what is the weather in sf'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-25', 'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area. ... Tuesday 06/25 ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719334068, 'localtime': '2024-06-25 9:47'}, 'current': {'last_updated_epoch': 1719333900, 'last_updated': '2024-06-25 09:45', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.

### Adding in memory

 this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [22]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im manoj!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gdc2', 'function': {'arguments': '{"query":"hi im manoj"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 965, 'total_tokens': 1038, 'completion_time': 0.057781903, 'prompt_time': 0.154014808, 'queue_time': None, 'total_time': 0.211796711}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_c4a72fb330', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1efda9a6-0b63-4e21-9f8c-c86996caffea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi im manoj'}, 'id': 'call_gdc2'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.optionpundit.com/moneymonday", "content": "Meet Your Host Hi I\'m Manoj Kumar B.Tech. MBA, CTA Since 2006, Manoj has trained thousands of people to start their trading journey, generate consistent returns from the financia

In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Manoj!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 1253, 'total_tokens': 1260, 'completion_time': 0.004789165, 'prompt_time': 0.190961158, 'queue_time': None, 'total_time': 0.195750323}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-02a40055-2c19-45d6-a8a2-942d72ae6076-0')]}}
----


In [26]:
# To start a new conversation we just need to change the thread id

config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8fsh', 'function': {'arguments': '{"query":"your name"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 966, 'total_tokens': 1044, 'completion_time': 0.061535294, 'prompt_time': 0.156476727, 'queue_time': None, 'total_time': 0.218012021}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4704dbda-0cb5-4210-b195-e9964dd77322-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'your name'}, 'id': 'call_8fsh'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/Your_Name", "content": "In its first week, the Blu-ray standard edition sold 202,370 units, the collector\'s edition sold 125,982 units and the special edition sold 94,079 units.[49] The DVD Standard Edition 